# In Silico Demonstration of the Veto Mechanism

By Charles Xu @ Caltech, 2025

For NB 150

## Set up the environment

We will use a biophysical neuron model constructed and optimized by the Allen institute through the Allen SDK, which builds on NEURON.

### Build NEURON from source

This will take quite a few minutes.

In [1]:
# Install dependencies
!apt-get install -y cmake build-essential libncurses-dev libreadline-dev \
  python3-dev libopenmpi-dev libhdf5-openmpi-dev git flex bison

# Clone the NEURON repository
!git clone https://github.com/neuronsimulator/nrn.git
%cd nrn

# Create build directory
!mkdir build
%cd build

# Configure the build with CMake
!cmake .. -DNRN_ENABLE_CORENEURON=OFF -DNRN_ENABLE_PYTHON=ON \
  -DNRN_ENABLE_MPI=OFF -DNRN_ENABLE_INTERVIEWS=OFF -DCMAKE_INSTALL_PREFIX=$HOME/nrn-install

# Build and install
!make -j4
!make install

# Add NEURON executables to shell PATH
import os
os.environ["PATH"] = f"{os.environ['HOME']}/nrn-install/bin:" + os.environ["PATH"]

# # Add to system PATH for ! commands
# %env PATH=$HOME/nrn-install/bin:$PATH

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libreadline-dev is already the newest version (8.1.2-1).
libreadline-dev set to manually installed.
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
libncurses-dev is already the newest version (6.3-2ubuntu0.1).
libncurses-dev set to manually installed.
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following additional packages will be installed:
  libfl-dev libfl2 libhdf5-openmpi-103-1 libhdf5-openmpi-cpp-103-1
  libhdf5-openmpi-fortran-102 libhdf5-openmpi-hl-100
  libhdf5-openmpi-hl-cpp-100 libhdf5-openmpi-hl-fortran-100
Suggested packages:
  bison-doc flex-doc libhdf5-doc
The following NEW packages will be installed:
  bison flex libfl-d

In [ ]:
import sys
sys.path.append(f"{os.environ['HOME']}/nrn-install/lib/python")

In [9]:
# Verify installation
!nrniv --version

# Verify Python binding
import neuron
print(f"NEURON Python binding is installed successfully ({neuron.__version__})")

NEURON -- VERSION 9.0a-1381-g04a50e981 master (04a50e981) 2025-04-17
NEURON Python binding is installed successfully (9.0a-1381-g04a50e981)


In [40]:
!pip install allensdk

In [30]:
!pip install matplotlib

In [35]:
!pip install --upgrade --force-reinstall numpy

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.5 which is incompatible.
allensdk 2.16.2 requires numpy<1.24, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.3 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
numba 0.60.0 requires

In [39]:
import numpy as np
print(np.__version__)

2.0.2


## Import model

In [21]:
%cd /content

# Downlaod the model from: https://github.com/CharlesXu-Jingyue/NB150_TA/raw/refs/heads/main/neuronal_model_497232918.zip
!wget https://github.com/CharlesXu-Jingyue/NB150_TA/raw/refs/heads/main/neuronal_model_497232918.zip

# Unzip
!unzip -o neuronal_model_497232918.zip -d neuronal_model

/content
--2025-04-20 22:41:23--  https://github.com/CharlesXu-Jingyue/NB150_TA/raw/refs/heads/main/neuronal_model_497232918.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CharlesXu-Jingyue/NB150_TA/refs/heads/main/neuronal_model_497232918.zip [following]
--2025-04-20 22:41:23--  https://raw.githubusercontent.com/CharlesXu-Jingyue/NB150_TA/refs/heads/main/neuronal_model_497232918.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241378 (236K) [application/zip]
Saving to: ‘neuronal_model_497232918.zip’

neuronal_model_4972 100%[===================>] 235.72K  --.-KB/s    in 0.04s   

2025-0

In [22]:
# Compile the model
!nrnivmodl neuronal_model/modfiles

/content
cfiles =
Mod files: "neuronal_model/modfiles/CaDynamics.mod" "neuronal_model/modfiles/Ca_HVA.mod" "neuronal_model/modfiles/Ca_LVA.mod" "neuronal_model/modfiles/Ih.mod" "neuronal_model/modfiles/Im.mod" "neuronal_model/modfiles/Im_v2.mod" "neuronal_model/modfiles/Kd.mod" "neuronal_model/modfiles/K_P.mod" "neuronal_model/modfiles/K_T.mod" "neuronal_model/modfiles/Kv2like.mod" "neuronal_model/modfiles/Kv3_1.mod" "neuronal_model/modfiles/Nap.mod" "neuronal_model/modfiles/NaTa.mod" "neuronal_model/modfiles/NaTs.mod" "neuronal_model/modfiles/NaV.mod" "neuronal_model/modfiles/SK.mod"

Creating 'x86_64' directory for .o files.

MODOBJS= ./CaDynamics.o ./Ca_HVA.o ./Ca_LVA.o ./Ih.o ./Im.o ./Im_v2.o ./Kd.o ./K_P.o ./K_T.o ./Kv2like.o ./Kv3_1.o ./Nap.o ./NaTa.o ./NaTs.o ./NaV.o ./SK.o
 -> Compiling mod_func.cpp
 -> NMODL ../neuronal_model/modfiles/CaDynamics.mod
Translating CaDynamics.mod into /content/x86_64/CaDynamics.cpp
 -> NMODL ../neuronal_model/modfiles/Ca_HVA.mod
 -> NMODL ../neuro

Now, we will load the compiled mechanisms.

In [23]:
from neuron import h, gui

h.nrn_load_dll("./x86_64/.libs/libnrnmech.so")

1.0

In [42]:
from neuron import h

# Create a section
soma = h.Section(name='soma')

# Now you can call psection
print(h.psection())

soma { nseg=1  L=100  Ra=35.4
	/*location 0 attached to cell 0*/
	/* First segment only */
	insert morphology { diam=500}
	insert capacitance { cm=1}
}
1.0


In [ ]:
!python -m allensdk.core.cell_types_cache.workflow \
    --manifest-file cell_types/manifest.json \
    --action get-ephys-data \
    --cell-specimen-id 497232918

In [59]:
!cd neuronal_model && python -m allensdk.model.biophysical.runner ./manifest.json

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/allensdk/model/biophysical/runner.py", line 240, in <module>
    run(vars(schema))
  File "/usr/local/lib/python3.11/dist-packages/allensdk/model/biophysical/runner.py", line 73, in run
    prepare_nwb_output(description.manifest.get_path('stimulus_path'),
  File "/usr/local/lib/python3.11/dist-packages/allensdk/model/biophysical/runner.py", line 163, in prepare_nwb_output
    copy(nwb_stimulus_path, nwb_result_path)
  File "/usr/lib/python3.11/shutil.py", line 431, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/usr/lib/python3.11/shutil.py", line 256, in copyfile
    with open(src, 'rb') as fsrc:
         ^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './475057895_ephys.nwb'
